In [1]:
# Import Required Libraries.

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

#regression model
from lightgbm import LGBMRegressor

In [4]:
# Import Preprocessed Data.

In [6]:
data = pd.read_csv(r'E:\autos1.csv')
data.head()

,Unnamed: 0,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,1,18300,coupe,2011,manual,190,not-declared,125000,5,diesel,audi,Yes
1,2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,not-declared
2,5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes
3,6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,No
4,8,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,not-declared


In [7]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(data[i])
    tr = mapper[i].transform(data[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    data.loc[:, i+'_labels'] = pd.Series(tr, index=data.index)
    
labeled = data[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [8]:
# Different Metrics Evaluation.

In [9]:

def find_scores(Y_actual, Y_pred, X_train):
    scores = dict()
    mae = mean_absolute_error(Y_actual, Y_pred)
    mse = mean_squared_error(Y_actual, Y_pred)
    rmse = np.sqrt(mse)
    rmsle = np.log(rmse)
    r2 = r2_score(Y_actual, Y_pred)
    n, k = X_train.shape
    adj_r2_score = 1 - ((1-r2)*(n-1)/(n-k-1))
    
    scores['mae']=mae
    scores['mse']=mse
    scores['rmse']=rmse
    scores['rmsle']=rmsle
    scores['r2']=r2
    scores['adj_r2_score']=adj_r2_score
    
    return scores

In [10]:
# Train Test Split.

In [11]:
X = labeled.iloc[:,1:].values
Y = labeled.iloc[:,0].values.reshape(-1,1)

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [13]:
# Predictive Modeling.

In [14]:
model = LGBMRegressor(boosting_type="gbdt",learning_rate=0.07,metric="rmse",n_estimators=300,objective="root_mean_squared_error",random_state=42,reg_sqrt=True)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

find_scores(Y_test, Y_pred, X_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'mae': 1586.200865351981,
 'mse': 9135171.967975652,
 'rmse': 3022.4447005653637,
 'rmsle': 8.013821286396137,
 'r2': 0.8763338009014708,
 'adj_r2_score': 0.8763230940566458}

In [15]:
# Save Model.

In [16]:
pickle.dump(model, open('Car resale Value Prediction.sav', 'wb'))